In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [912 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
#Download Postgress driver to connect
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-30 22:12:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.98MB/s    in 0.2s    

2022-08-30 22:12:45 (4.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big_Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-08-31 00:00:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [5]:
#Count the number of records (rows) in the dataset.
df.count()

4366916

In [6]:
#Create review_id_table w/ columns review_id, customer_id, product_id, product_parent, review_date
review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781|2015-08-31 00:00:00|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853|2015-08-31 00:00:00|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660|2015-08-31 00:00:00|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263|2015-08-31 00:00:00|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171|2015-08-31 00:00:00|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457|2015-08-31 00:00:00|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602|2015-08-31 00:00:00|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055|2015-08-31 00:00:00|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438|2015-08-31 00:00:00|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213|2015-08-31 00:00:00|
+--------------+---------

In [7]:
#Create products_table w/product_id, product_title unique values
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CJL5ES|L.B. Evans Men's ...|
|B002MCVHUQ|New Balance Insol...|
|B00UXBFW6S|Kenox Sling Backp...|
|B002CMM4M6|Capezio Women's 1...|
|B00HNO32BU|Timberland PRO Me...|
|B00C6BQQW2|New Balance Men's...|
|B00M0NREE0|New Balance Women...|
|B00ZC4VIKK|Donalworld Women ...|
|B00291CGH8|Justin Boots Wome...|
|B000P6GKOO|Nike Men's Rosher...|
+----------+--------------------+
only showing top 10 rows



In [8]:
#Create customers table w customer_id and customer_count
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df.show(10)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   16121565|                 5|
|   48146680|                 1|
|   11062912|                 1|
|   51451778|                 1|
|   27920838|                 1|
|    4919528|                 3|
|   47802851|                 2|
|   42560427|                 1|
|   29467780|                 1|
|   49703087|                12|
+-----------+------------------+
only showing top 10 rows



In [9]:
#Create vine_table w/ review_id, star_rating, helpful_votes, total_votes, vine
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|
|R12VVR0WH5Q24V|          5|            0|          0|   N|
| RNCCKB6TV5EEF|          4|            0|          0|   N|
|R2NZXYIVCGB13W|          5|            0|          6|   N|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|
|R12ENYLFGGNWRV|          5|            1|          1|   N|
|R2R07E5PNXEUO5|          4|            0|          0|   N|
|R27BA52AKWMWN3|          5|            0|          0|   N|
| RLF8DOID2KD5O|          3|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [10]:
#Write dataframe to database
mode = "append"
jdbc_url="jdbc:postgresql://{endpoint}:5432/postgres"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
#Review_id to table
review_id.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [12]:
#Product_df to table
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
#Customer_df to table
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [14]:
#Vine_df to table
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)